In [1]:
!pip install librosa matplotlib spafe torch pandas

You should consider upgrading via the '/home/ashutosh/.pyenv/versions/3.8.13/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
#run
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import defaultdict
from spafe.utils import vis
from spafe.features.lfcc import lfcc
import pandas as pd

In [3]:
import torch
import torch.nn as nn
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SelfAttention(nn.Module):

  def __init__(self, embed_size, num_heads):

    super(SelfAttention, self).__init__()

    self.embed_size = embed_size
    self.num_heads = num_heads
    self.head_dim = embed_size // num_heads

    assert (self.head_dim * num_heads == embed_size), "Embed size needs to be divisible by heads"

    self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.fc_out = nn.Linear(num_heads * self.head_dim, embed_size)


  def forward(self, values, keys, query, mask):
    N = query.shape[0]
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

    # Split embedding into self.num_heads pieces
    values = values.reshape(N, value_len, self.num_heads, self.head_dim)
    keys = keys.reshape(N, key_len, self.num_heads, self.head_dim)
    queries = query.reshape(N, query_len, self.num_heads, self.head_dim)

    values = self.values(values)
    keys = self.keys(keys)
    queries = self.queries(queries)

    energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]) # MatMul Q and K
    # queries shape: (N, query_len, heads, heads_dim)
    # keys shape: (N, query_len, heads, heads_dim)
    # energy shape: (N, heads, query_len, key_len)

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float("-1e20"))

    attention = torch.softmax(energy / (self.embed_size ** 0.5), dim = 3)

    out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
      N, query_len, self.num_heads * self.head_dim
    )
 
    out = self.fc_out(out)

    return out


In [4]:
class PositionalEncoding(nn.Module):

  def __init__(self, embed_size, dropout, max_len = 5000):
    super().__init__()

    self.dropout = nn.Dropout(p = dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embed_size, 2) * (-math.log(10000.0) / embed_size))
    self.position_encoding = torch.zeros(max_len, 1, embed_size).to(device)
    self.position_encoding[:, 0, 0::2] = torch.sin(position * div_term).to(device)
    self.position_encoding[:, 0, 1::2] = torch.cos(position * div_term).to(device)
    self.register_buffer('pe', self.position_encoding)

  def forward(self, x):
    x = x + self.position_encoding[:x.size(0)]
    return self.dropout(x)

In [5]:
class TransformerBlock(nn.Module):

  def __init__(self, embed_size, heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()

    self.attention = SelfAttention(embed_size, heads)
    self.norm1 = nn.LayerNorm(embed_size)
    self.norm2 = nn.LayerNorm(embed_size)

    self.feed_forward = nn.Sequential(
      nn.Linear(embed_size, forward_expansion * embed_size),
      nn.ReLU(),
      nn.Linear(forward_expansion * embed_size, embed_size)
    )

    self.dropout = nn.Dropout(dropout)


  def forward(self, value, key, query, mask):
    attention = self.attention(value, key, query, mask)

    x = self.dropout(self.norm1((attention + query)))

    forward = self.feed_forward(x)
    out = self.dropout(self.norm2(x + forward))

    return out

In [6]:
class Encoder(nn.Module):

  def __init__(
    self,
    src_vocab_size,
    embed_size,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    max_length
  ):
    super(Encoder, self).__init__()
    self.embed_size = embed_size
    self.device = device
    self.position_embedding = PositionalEncoding(embed_size, dropout, src_vocab_size)

    self.layers = nn.ModuleList(
      [
        TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)
      ]
    )

    self.dropout = nn.Dropout(dropout)


  def forward(self, x, mask):
    N, seq_length = x.shape

    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

    out = self.dropout(self.position_embedding(positions))

    for layer in self.layers:
      out = layer(out, out, out, mask)

    return out


In [7]:
class Transformer(nn.Module):

  def __init__(
    self,
    src_vocab_size,
    src_pad_index,
    embed_size = 256,
    num_layers = 6,
    forward_expansion = 4,
    heads = 8,
    dropout = 0,
    device = "cuda",
    max_length = 500
  ):
    super(Transformer, self).__init__()

    self.encoder = Encoder(
      src_vocab_size, embed_size, num_layers, heads,
      device, forward_expansion, dropout, max_length)

    self.src_pad_index = src_pad_index
    self.device = device


  def make_src_mask(self, src):
    src_mask = (src != self.src_pad_index).unsqueeze(1).unsqueeze(2)

    # (N, 1, 1, src_len)
    return src_mask.to(self.device)

  
  def forward(self, src):
    src_mask = self.make_src_mask(src)
    encoder_output = self.encoder(src, src_mask)
    
    return encoder_output

In [8]:
from torch.utils.data import random_split, Dataset, DataLoader

class AudioFeatureDataset(Dataset):
    def __init__(self, annotations_file, mode = 'train'):
        self.features = pd.read_csv(annotations_file)
        self.features = self.features.drop(['Unnamed: 0'], axis = 1)
        self.train_subset, self.valid_subset = random_split(self.features, [0.8, 0.2])
        self.x_train = torch.Tensor(self.train_subset.dataset.drop([
            'class', str(len(self.train_subset.dataset.columns) - 2)
        ], axis = 1).values)
        self.y_train = torch.Tensor(self.train_subset.dataset['class'].values)
        self.x_valid = torch.Tensor(self.valid_subset.dataset.drop([
            'class', str(len(self.train_subset.dataset.columns) - 2)], axis = 1).values)
        self.y_valid = torch.Tensor(self.valid_subset.dataset['class'].values)
        self.mode = mode
        
    def __len__(self):
        if self.mode == 'train':
            return self.x_train.shape[0]

        return self.x_valid.shape[0]

    def __getitem__(self, idx):
        if self.mode == 'train':
            return self.x_train[idx], self.y_train[idx]
        
        return self.x_valid[idx], self.y_valid[idx]

In [9]:
training_dataset = AudioFeatureDataset('./DatabaseDistorted/final_tii_all_finaldata/features/2075_concatenate_dm_4featu.csv')
validation_dataset = AudioFeatureDataset(
  './DatabaseDistorted/final_tii_all_finaldata/features/2075_concatenate_dm_4featu.csv',
  mode = 'valid'
)

training_dataloader = DataLoader(training_dataset, batch_size = 2, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 2, shuffle = True)

In [10]:
num_layers = 4
src_vocab_size = X_train.shape[1]
src_pad_index = 0
embed_size = X_train.shape[2]
num_heads = 4
dropout = 0.1
output_size = 1
batch_size = 64

model = Transformer(
  src_vocab_size,
  src_pad_index,
  embed_size = embed_size,
  dropout = dropout,
  heads = num_heads,
  num_layers = num_layers,
  ).to(device)

NameError: name 'X_train' is not defined

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000003)

loss_fn = nn.MSELoss()

running_loss = 0.
last_loss = 0.

model.train()

epochs = 1000

for epoch in range(epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  for batch, (X, y) in enumerate(training_dataloader):
    # Compute prediction and loss
    X = X.to(device)
    y = y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
  
  
  model.eval()
  size = len(validation_dataloader.dataset)
  num_batches = len(validation_dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in validation_dataloader:
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

print('Done')

Epoch 1
-------------------------------
0


RuntimeError: The size of tensor a (296) must match the size of tensor b (2) at non-singleton dimension 3

In [ ]:
# References:
# 1. https://www.youtube.com/watch?v=zxQyTK8quyY
# 2. https://www.youtube.com/watch?v=bCz4OMemCcA
# 3. https://www.youtube.com/watch?v=gJ9kaJsE78k